In [ ]:
import keras
from sklearn.model_selection import train_test_split
# ^^^ pyforest auto-imports - don't write above this line
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
GPU_ID= '0'
os.environ["CUDA_VISIBLE_DEVICES"] =GPU_ID
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

In [ ]:
#import keras
from keras import backend as K
import numpy as np
import gdal

In [ ]:
validation_states = ['ROIs13_high_roshan_s2']
data_dir = "SEN12MS/processed"

In [ ]:
validation_patches=[]
for state in validation_states:
    fn= os.path.join(data_dir,'%s_val_patches/' %state)
    #print(fn)
    with os.scandir(fn) as entries:
        for entry in entries:
            #print(entry.name)
            with os.scandir(fn+entry.name) as folders:
                for folder in folders:
                    #print(folder.name)
                    with os.scandir(fn+entry.name+'/'+folder.name) as files:
                        for file in files:
                            #print(file.name)
                            validation_patches.append((os.path.join(fn+entry.name+'/'+folder.name+'/',file.name),state))

In [ ]:
train_patches=[]
train_states = ['dfc_1']
for state in train_states:
    fn= os.path.join(data_dir,'%s/' %state)
    #print(fn)
    with os.scandir(fn) as entries:
        for entry in entries:
            #print(entry.name)
            with os.scandir(fn+entry.name) as folders:
                for folder in folders:
                    #print(folder.name)
                    with os.scandir(fn+entry.name+'/'+folder.name) as files:
                        for file in files:
                            #print(file.name)
                            train_patches.append((os.path.join(fn+entry.name+'/'+folder.name+'/',file.name),state))

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

In [ ]:
print(validation_patches[1][0])

In [ ]:
#one by one checking
iou=0
f1=0
for i in range(len(train_patches)):
    original = np.load(validation_patches[i][0])
    tar_img= np.expand_dims(original[:,:,5],axis=0)
    tar_img=keras.utils.to_categorical(tar_img, 11)
    
    ds = gdal.Open(train_patches[i][0])
    if ds != None:
        outp= ds.ReadAsArray()
        outp[outp>= 10.] = 10
        outp[outp<= 1.] = 1
        #print(outp.shape)
        outp=np.expand_dims(keras.utils.to_categorical(outp, 11),axis=0)
        iou = iou + K.get_value(iou_coef(tar_img,outp))
        f1 = f1 + K.get_value(dice_coef(tar_img,outp))
        ds =None
        #ds.close()
        print("Done with ",str(i))
    else:
        print("error with gdal opening file.")

a = f1/len(validation_patches)
print("The f1 score is ",a)
b = iou/len(validation_patches)
print("The iou score is ",b)